In [13]:
import cv2
import numpy as np
import pandas as pd
import pickle as pkl
import imageio
from matplotlib import pyplot as plt
from optical_flow import *
from tqdm import tqdm
from tracking_utils import *
import matplotlib.animation as animation
import pickle as pkl
from load_utils import *
from eval_utils import *
from sort import *
from VehicleDetection import *

In [14]:
# SELECT SEQUENCE TO EVALUATE
S = 'seq3'

# SELECT CAMERA TO EVALUATE
C = 'c10'

# SET NUMBER OF FRAMES
num_frames = 2141

In [15]:
# Load GT detections
gt_detect_path = f'./cam_pred/{S}_{C}_gt.pkl'

# Load computed detections
detection_path = f'./cam_pred/{S}_{C}.pkl'

with open(detection_path, 'rb') as f:
    all_detections = pkl.load(f)

with open(gt_detect_path, 'rb') as f:
    all_gt_detections = pkl.load(f)


In [16]:
# Load video frames
results = {}

mot_tracker = Sort(max_age=10, min_hits=3, iou_threshold=0.3) # Sort Kalman tracker with default values

# Iterate Frames
for frame in tqdm(range(num_frames), desc = "Tracking objects each frame..."):
    dets = []

    detections = get_detection_dataframe(all_detections[str(frame)], iclLineAndUpdate = False, firstFrame = True).sort_values("track") # Load detections

    for (track_id, det, bbox, size, colour) in detections.itertuples(index=False): # Iter All Detections
        dets.append(np.array(bbox))


    trackers = mot_tracker.update(np.array(dets)) # Update tracker with current detections


    for d in trackers: # Store new bboxes
        d = d.astype(np.int32)
        if frame not in results:
            results[frame] = {d[4] % 47: {"bbox": [d[0], d[1], d[2], d[3]]}}
        else:
            results[frame][d[4] % 47] = {"bbox": [d[0], d[1], d[2], d[3]]}

            
# Save Results to Disk
pkl.dump(results, open(f"sort_bbox_{S}_{C}.pkl", "wb"))

Tracking objects each frame...: 100%|██████████| 2141/2141 [00:13<00:00, 162.69it/s]


In [17]:
# Pkl to pd.DataFrame
colours = np.random.rand(47, 3) #Generate Random Colors
old_pkl = pkl.load(open(f"sort_bbox_{S}_{C}.pkl", "rb"))
new_pkl = []
df = None

for frame in old_pkl.keys():
    new_data = {"frame": [],"track": [], "bbox": [], "colour": [], "size": [], "detection": []}
    old_data = old_pkl[frame]
    
    for track_id in old_data.keys():
        new_data["track"].append(track_id)
        bbox = old_data[track_id]["bbox"]
        new_data["bbox"].append(bbox)
        new_data["frame"].append("Frame {}".format(frame))
        new_data["colour"].append(np.round(colours[track_id]*255).astype(np.uint8))
        new_data["size"].append(int(np.abs(bbox[0] - bbox[2]) * np.abs(bbox[1] - bbox[3])))
        new_data["detection"].append(VehicleDetection(frame=frame, ID=track_id, width=np.abs(bbox[0] - bbox[2]), conf=0.5,
                                                      height=np.abs(bbox[1] - bbox[3]), left=bbox[0], right=bbox[2], top=bbox[1], bot=bbox[3]))
    
    df = pd.DataFrame.from_dict(new_data).sort_values(by="track")

    new_pkl.append(df)

pkl.dump(new_pkl, open(f"kalman_tracking_{S}_{C}.pkl", "wb"))

In [76]:
detections = {}
with open(f'kalman_tracking_{S}_{C}.pkl','rb') as openFile:
    detections = pkl.load(openFile)

## Evaluation

In [77]:
total_frames = num_frames
initial_frame = 1

In [47]:
detections[7]
# detections[7].at[1, 'track']
# len(detections[7])

,frame,track,bbox,colour,size,detection
11,Frame 0,0,"[541, 74, 677, 192]","[58, 17, 88]",16048,"Frame 0, TL [541,74], BR [677,192], Confidence..."
10,Frame 0,1,"[906, 69, 1031, 163]","[142, 47, 251]",11750,"Frame 0, TL [906,69], BR [1031,163], Confidenc..."
9,Frame 0,2,"[474, 217, 525, 325]","[63, 200, 78]",5508,"Frame 0, TL [474,217], BR [525,325], Confidenc..."
8,Frame 0,3,"[1277, 349, 1540, 566]","[176, 234, 9]",57071,"Frame 0, TL [1277,349], BR [1540,566], Confide..."
7,Frame 0,4,"[528, 63, 756, 205]","[240, 40, 204]",32376,"Frame 0, TL [528,63], BR [756,205], Confidence..."
6,Frame 0,5,"[483, 223, 524, 278]","[6, 84, 137]",2255,"Frame 0, TL [483,223], BR [524,278], Confidenc..."
5,Frame 0,6,"[477, 242, 516, 351]","[141, 87, 35]",4251,"Frame 0, TL [477,242], BR [516,351], Confidenc..."
4,Frame 0,7,"[1127, 308, 1561, 587]","[4, 230, 7]",121086,"Frame 0, TL [1127,308], BR [1561,587], Confide..."
3,Frame 0,8,"[480, 43, 1129, 321]","[192, 70, 202]",180422,"Frame 0, TL [480,43], BR [1129,321], Confidenc..."
2,Frame 0,9,"[464, 220, 538, 288]","[26, 176, 229]",5032,"Frame 0, TL [464,220], BR [538,288], Confidenc..."


In [85]:
i = initial_frame

while i < total_frames:
    for index, previous_detection in detections[i-1].iterrows():
        length, _ = detections[i].shape

        td = []
        for j in range(length):
            IoU = previous_detection['detection'].IoU(detections[i].iloc[j]['detection'])
            if IoU > 0.95:
                td.append(j)
                
        detections[i] = detections[i].drop(td)
    i += 1

KeyError: "['0'] not found in axis"

In [74]:
detections[6]

,frame,track,bbox,colour,size,detection
7,Frame 6,0,"[540, 74, 676, 192]","[58, 17, 88]",16048,"Frame 6, TL [540,74], BR [676,192], Confidence..."
6,Frame 6,1,"[906, 69, 1030, 163]","[142, 47, 251]",11656,"Frame 6, TL [906,69], BR [1030,163], Confidenc..."
5,Frame 6,2,"[467, 216, 519, 325]","[63, 200, 78]",5668,"Frame 6, TL [467,216], BR [519,325], Confidenc..."
4,Frame 6,3,"[1274, 348, 1541, 567]","[176, 234, 9]",58473,"Frame 6, TL [1274,348], BR [1541,567], Confide..."
3,Frame 6,4,"[527, 63, 758, 205]","[240, 40, 204]",32802,"Frame 6, TL [527,63], BR [758,205], Confidence..."
2,Frame 6,5,"[478, 220, 520, 286]","[6, 84, 137]",2772,"Frame 6, TL [478,220], BR [520,286], Confidenc..."
1,Frame 6,7,"[1123, 308, 1558, 586]","[4, 230, 7]",120930,"Frame 6, TL [1123,308], BR [1558,586], Confide..."
0,Frame 6,8,"[424, 41, 1046, 323]","[192, 70, 202]",175404,"Frame 6, TL [424,41], BR [1046,323], Confidenc..."


In [35]:
acc = create_accumulator()

for frame in tqdm(range(initial_frame, total_frames-1)):
    if str(frame) in all_gt_detections:
        acc, frame_id = tracking_acc(frame, all_gt_detections, detections, acc)

print(f'{S} {C} results:')
display_metrics(acc)

100%|██████████| 2139/2139 [00:28<00:00, 75.18it/s] 

seq3 c10 results:
     num_frames  precision  recall       idp  idr      idf1
acc        1355   0.132571     1.0  0.132571  1.0  0.234106
